In [21]:
import json
import time

import pandas as pd
from tqdm import tqdm
import tweepy
from dotenv import load_dotenv
import os

from tweepy import TweepyException

from utils import divide_chunks

load_dotenv()  # take environment variables from .env.

True

In [18]:
client = tweepy.Client(bearer_token=os.environ['BEARER'],
                       wait_on_rate_limit=True,)
# with API rather than client, could add                        retry_count=10, retry_delay=60, retry_errors=set([503])

In [9]:
DATA_PATH = 'data'
YEARS = [2016, 2020]
ids = set()
for year in YEARS:
    with open(os.path.join(DATA_PATH, f'follower_lists_{year}.jsonl')) as f:
        for l in f:
            for _, ids_ in json.loads(l).items():
                ids.update(ids_)
ids = list(ids)

In [10]:
# DATA_PATH = 'data'
# users_path = os.path.join(DATA_PATH, 'followers_2016.jsonl')
# with open(users_path, encoding='utf8') as f:
#     users = pd.concat(
#         pd.DataFrame([dict(follower, **{'follows':pollster}) for follower in followers])
#         for l in f for pollster, followers in json.loads(l).items()
#     )
# users.head()

In [11]:
# # Replace User IDs
# ids = users.id.tolist()
# ids[:3]

In [12]:
len(ids)

5458196

In [22]:
errors = list()

In [23]:
out_path = os.path.join(DATA_PATH, 'followers_rehydrated.jsonl')
already_scraped = set()
if os.path.exists(out_path):
    with open(out_path) as f:
        already_scraped = {json.loads(l)['id'] for l in f}
print(f"found {len(already_scraped)} users")

with open(out_path, 'a+', encoding='utf8') as f:
    ids_ = [id_ for id_ in ids if id_ not in already_scraped]
    for chunk in tqdm(divide_chunks(ids_, 100), desc='batch', total = len(ids_)//100+bool(len(ids_)%100)):
        user_fields = ['created_at','description','entities','id','location','name','pinned_tweet_id','profile_image_url','protected','public_metrics','url','username','verified','withheld']
        try:
            returned = client.get_users(ids=chunk, user_fields=user_fields)
            for user in returned.data:
                f.write(json.dumps(user.data)+'\n')
        except TweepyException as e:
            print(e)
            errors.extend(chunk)
            time.sleep(60)


found 1097530 users


batch:  28%|██▊       | 15505/54582 [12:47:18<3:47:31,  2.86it/s]    

503 Service Unavailable
Service Unavailable


batch:  65%|██████▌   | 35541/54582 [29:33:47<2:57:13,  1.79it/s]    

503 Service Unavailable
Service Unavailable


batch:  87%|████████▋ | 47368/54582 [39:19:39<39:30,  3.04it/s]     

503 Service Unavailable
Service Unavailable


batch: 100%|██████████| 54582/54582 [45:20:02<00:00,  2.99s/it]    


In [24]:
len(errors)

300

In [26]:
with open(out_path, 'a', encoding='utf8') as f:
    for chunk in tqdm(divide_chunks(errors, 100), desc='batch', total = len(errors)//100+bool(len(errors)%100)):
        user_fields = ['created_at','description','entities','id','location','name','pinned_tweet_id','profile_image_url','protected','public_metrics','url','username','verified','withheld']
        try:
            returned = client.get_users(ids=chunk, user_fields=user_fields)
            for user in returned.data:
                f.write(json.dumps(user.data)+'\n')
        except TweepyException as e:
            print(e)
            errors.extend(chunk)
            time.sleep(60)

batch: 100%|██████████| 3/3 [00:00<00:00,  3.21it/s]


In [27]:
already_scraped = set()
if os.path.exists(out_path):
    with open(out_path) as f:
        already_scraped = {json.loads(l)['id'] for l in f}
print(f"found {len(already_scraped)} users")

found 5456127 users
